## Парсинг данных. HTML, Beautiful Soap

### Задание.
Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру.  
Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:  
- Наименование вакансии. Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).  
- Ссылку на саму вакансию. 
- Сайт, откуда собрана вакансия.  

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd
from time import sleep

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
          }
URL_HH_RU = 'https://hh.ru/vacancies/matematik'
#URL_RABOTA_RU = 'https://rabota.ru/'
#URL_SUPER_JOB = 'https://www.superjob.ru//'

response = requests.get(url=URL_HH_RU, headers=headers)
response.url

'https://hh.ru/vacancies/matematik'

In [6]:
num_page = 0
list_dom = []
for page in range(2):
    response = requests.get(url = URL_HH_RU, headers = headers)
    num_page +=1
    print(response.status_code)
    dom = bs(response.text, 'html.parser')
    tag_vacancy = dom.find_all('div', class_= 'serp-item')
    print(len(tag_vacancy))
    list_dom.append(tag_vacancy)
    sleep(1)

200
50
200
50


In [7]:
number = 1
vacancy_dict = {}
site_name = 'hh.ru'
for dom in list_dom:
  for vacancy in tag_vacancy:
    vacancy_title = vacancy.find('a', attrs={'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}).string
    if vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}) == None:
      min_compensation = None
      max_compensation = None
      currency = None
    else:
      compensation = vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}).text.replace(' ', '').split()
      if compensation[0] == 'от':
        min_compensation = int(compensation[1])
        max_compensation = None
        currency = compensation[2]
      elif compensation[0] == 'до':
        min_compensation = None
        max_compensation = int(compensation[1])
        currency = compensation[2]
      else:
        min_compensation = int(compensation[0])
        max_compensation = int(compensation[2])
        currency = compensation[3]
    vacancy_link =  vacancy.find('a', attrs={'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}).get('href')
    vacancy_dict[number] = {
         'должность': vacancy_title,
         'минимальная зарплата' : min_compensation,
         'максимальная зарплата' : max_compensation,
         'валюта' : currency,
         'ссылка на вакансию' : vacancy_link,
         'сайт' : site_name
         }
    number +=1


In [8]:
vacancy_dict

{1: {'должность': 'Преподаватель математики/физики/информатики (подготовка к ЕГЭ)',
  'минимальная зарплата': 20000,
  'максимальная зарплата': 150000,
  'валюта': 'руб.',
  'ссылка на вакансию': 'https://hh.ru/vacancy/69430374?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA',
  'сайт': 'hh.ru'},
 2: {'должность': 'Программирующий математик',
  'минимальная зарплата': None,
  'максимальная зарплата': None,
  'валюта': None,
  'ссылка на вакансию': 'https://hh.ru/vacancy/54770574?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA',
  'сайт': 'hh.ru'},
 3: {'должность': 'Помощник преподавателя Математики',
  'минимальная зарплата': 15000,
  'максимальная зарплата': 45000,
  'валюта': 'руб.',
  'ссылка на вакансию': 'https://hh.ru/vacancy/69212519?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA',
  'сайт': 'hh.ru'},
 4: {'должность': 'Математик - программист C/Python',
  'минимальная зарплата': 90000,
  'максимальная зарплата': 140000,
  'валюта': 'руб.

In [47]:
vacancy_df = pd.DataFrame(vacancy_dict).T
vacancy_df

,должность,минимальная зарплата,максимальная зарплата,валюта,ссылка на вакансию,сайт
1,Преподаватель математики/физики/информатики (п...,20000,150000,руб.,https://hh.ru/vacancy/69430374?query=%D0%BC%D0...,hh.ru
2,Помощник преподавателя Математики,15000,45000,руб.,https://hh.ru/vacancy/69212519?query=%D0%BC%D0...,hh.ru
3,Программирующий математик,None,None,None,https://hh.ru/vacancy/54770574?query=%D0%BC%D0...,hh.ru
4,Математик - программист C/Python,90000,140000,руб.,https://hh.ru/vacancy/69036019?query=%D0%BC%D0...,hh.ru
5,Частный учитель по математике и дз к ученику 6...,80000,None,руб.,https://hh.ru/vacancy/69232202?query=%D0%BC%D0...,hh.ru
...,...,...,...,...,...,...
96,Преподаватель математических дисциплин (колледж),None,None,None,https://hh.ru/vacancy/69463858?query=%D0%BC%D0...,hh.ru
97,Преподаватель математики,None,50000,руб.,https://hh.ru/vacancy/69559043?query=%D0%BC%D0...,hh.ru
98,Учитель математики и информатики,85000,None,руб.,https://hh.ru/vacancy/69511232?query=%D0%BC%D0...,hh.ru
99,Методист по математике,None,None,None,https://hh.ru/vacancy/69602746?query=%D0%BC%D0...,hh.ru


In [51]:
vacancy_df.to_csv('Vacancy_HH.csv', index=None)